In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
#                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
#                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                              ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [7]:
arr = ['Excuse Me' , 'Thank You'   ,'from'		  ,'jump',	 'sleep'	, 'what',	 'your', 'I',    'airplane',	  'hello'	  ,'my',	 'throw'	, 'where',
'I am Fine',   'clean'	  ,'how are you',   'name'	, 'walk',	 'you']
actions = list(arr)
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'Excuse Me': 0,
 'Thank You': 1,
 'from': 2,
 'jump': 3,
 'sleep': 4,
 'what': 5,
 'your': 6,
 'I': 7,
 'airplane': 8,
 'hello': 9,
 'my': 10,
 'throw': 11,
 'where': 12,
 'I am Fine': 13,
 'clean': 14,
 'how are you': 15,
 'name': 16,
 'walk': 17,
 'you': 18}

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from tensorflow import keras

In [9]:
log_dir = os.path.join('Logs')
tb_callback = [TensorBoard(log_dir=log_dir),EarlyStopping(monitor = 'categorical_accuracy', verbose = 1, restore_best_weights=True,mode="max",patience = 20)]

In [10]:
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(30,258)))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1'))
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(19, activation='softmax'))

In [11]:
b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
b_model.load_weights('96%.h5')

In [19]:
b_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 7740)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               3963392   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 19)                9747      
                                                                 
Total params: 3,973,139
Trainable params: 3,973,139
Non-trainable params: 0
_________________________________________________________________


In [13]:
from scipy import stats
colors = [(245,117,16), (117,245,16), (16,117,245), (20,100,150)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [18]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res = b_model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))


            #3. Viz logic
                if np.unique(predictions[-5:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] >= threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 7:
                    sentence = sentence[-7:]

                # Viz probabilities
#                 image = prob_viz(res, actions, image, colors)

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
except Exception as e:
    print(e)
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 68ms/step
your
1/1 [==============================] - 0s 38ms/step
your
1/1 [==============================] - 0s 47ms/step
your
1/1 [==============================] - 0s 54ms/step
your
1/1 [==============================] - 0s 34ms/step
your
1/1 [==============================] - 0s 43ms/step
your
1/1 [==============================] - 0s 41ms/step
your
1/1 [==============================] - 0s 43ms/step
your
1/1 [==============================] - 0s 34ms/step
your
1/1 [==============================] - 0s 51ms/step
your
1/1 [==============================] - 0s 40ms/step
your
1/1 [==============================] - 0s 47ms/step
your
1/1 [==============================] - 0s 50ms/step
your
1/1 [==============================] - 0s 36ms/step
your
1/1 [==============================] - 0s 52ms/step
your
1/1 [==============================] - 0s 44ms/step
your
1/1 [==============================] - 0s 47ms/step
your
1/1 [==========================